In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
import sys
import pandas as pd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
# from Bio import SeqIO 

# import sklearn


ModuleNotFoundError: No module named 'torch'

## Preprocessing the data
We are going to use similiar thinking to sentence construction.
https://chriskhanhtran.github.io/posts/cnn-sentence-classification/

Data from: https://europepmc.org/article/med/16122420

In [ ]:
def prepare_data(seqs):
    '''
    Given a list of sequences, will turn into a tokenized vector.
    
    ARGS:
        seqs: a list of strings where every string is a sequence
    RETURNS:
        tokenized_seqs (list(list(str))): list of list of tokens
        voc2ind (dict) a dictionary where keys are letters, values are the corresponding token
    '''
    max_len = 0
    
    # build up a voc2ind (letters:token)
    # based on ATGC and include padding and unknown tokens
    voc2ind = {voc:ind for ind,voc in enumerate(['<pad>', '<unk>', 'A', 'T', 'C', 'G'])}
    
    i = len(voc2ind)
    
    # tokenize the sequences
    tokenized_seqs = []
    for seq in seqs:
        tokenized_seq = []
        for e in seq:
            # make sure the sequence is upper case, a == A
            seq = seq.upper()
            # if we haven't seen this letter before, add to the corupus
            if not e in voc2ind:
                voc2ind[e] = i
                i += 1
            tokenized_seq.append(voc2ind[e])
        tokenized_seqs.append(tokenized_seq)
        
    return tokenized_seqs, voc2ind
        
res = prepare_data(['ATCG', 'TAGA', 'APO'])
print(res)
assert(res[0] == [[2, 3, 4, 5], [3, 2, 5, 2], [2, 6, 7]]), res[0]

In [ ]:
def pad(tokenized_seqs, voc2ind):
    '''
    Pad each sequence to the maximum length by adding a <pad> token
    
    ARGS:
        tokenized_seqs (list(list(str))): list of list of tokens
        voc2ind (dict) a dictionary where keys are letters, values are the corresponding token
    RETURNS:
        a numpy array of all the tokenized sequences that have been padded to be the same
        length.
    '''

    padded_seqs = []
    
    # find max sequence length
    max_len = 0
    for seq in tokenized_seqs:
        max_len = max(len(seq), max_len)
    
    # add padding so sequences are max_length
    for seq in tokenized_seqs:
        padded_seq = seq + [voc2ind['<pad>']] * (max_len - len(seq))
        padded_seqs.append(padded_seq)
        
    return np.array(padded_seqs)

In [ ]:
def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=50):
    """
    Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """

    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

## Splice Dataset
https://archive.ics.uci.edu/ml/datasets/Molecular+Biology+%28Promoter+Gene+Sequences%29

In [ ]:
# loading dataset
# all sequences put in seq
# all labels in corresponding index in labels
seqs = []
labels = []

f = open('splice.data')
for line in f:
    line = [i.replace(',', '') for i in line.split()]
    seqs.append(line[2])
    labels.append(line[0])
f.close()

# tokenizing and getting a vocab
tokenized_seqs, voc2ind = prepare_data(seqs)

# padding
tokenized_seqs = pad(tokenized_seqs, voc2ind)

# Showing the result of this:
tokenized_seqs, voc2ind, set(labels)

In [ ]:
# Train Test Split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_seqs, labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, test_dataloader = data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)